this code is a compilation of the different models we built to address question 2

For Question 2, keywords to predict subreddit

In [2]:
import pandas as pd
import os
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

In [3]:
csv_directory = '/Users/emilywu/Downloads/archive (1) copy 2' # replace with file directory

combined_data = pd.DataFrame()


# add all files in folder into one csv file for title cleaning  

for file_name in os.listdir(csv_directory):
    if file_name.endswith('.csv'):  
        file_path = os.path.join(csv_directory, file_name)

        current_data = pd.read_csv(file_path)

        combined_data = pd.concat([combined_data, current_data], ignore_index=True)

combined_data.to_csv('combined_file.csv', index=False)


In [4]:


# only columns of interest for this question are subreddit and title 
new_data = combined_data[['subreddit', 'title']]


# data cleaning drop all nas 
new_data['subreddit'].dropna()
new_data['title'].dropna()


# see how many entries are in the following subreddits 
new_data['subreddit'].value_counts()

new_data = new_data.dropna(subset=['subreddit'])

new_data['subredditId'] = new_data['subreddit'].factorize()[0]

category = new_data[['subreddit', 'subredditId']].drop_duplicates().sort_values('subredditId')
print(category)


           subreddit  subredditId
0     Showerthoughts            0
1013  wholesomememes            1
2000           funny            2
2988            pics            3
3984            gifs            4
4980           Jokes            5
5976          videos            6
6974            tifu            7
7968           memes            8
8949             aww            9
9942            IAmA           10


In [5]:
import re

stop_words = {
    'is', 'a', 'this', 'another', 'the', 'to', 'i', 'from', 'at', 'been', 'my', 
    'by', 'for', 'that', 'our', 'after', '[oc]', 'you', 'your', 'how', 'see', 'where', 
    'what', 'any', 'or', 'we', 'they', 'if', 'her', 'him', 'he', 'so', 'but', '2019', 
    '2020', '2021', '2016', 'x', 'his', 'are', 'just', 'do', 'will', 'be', 'us', 
    'their', 'who', 'when', 'just', 'IAMA', 'ama', "eli5", "wp", "tifu", 'iama', 'lpt'
}


# function to make all words lowercase, and clean up punctuation
def preprocess_title(title):

    title = re.sub(r'[^\w\s]', '', title)

    words = title.split()
    
    filtered_words = []  

    for word in words:
        lower_word = word.lower()  
        if lower_word not in stop_words: 
            filtered_words.append(lower_word)  

    return ' '.join(filtered_words)



In [6]:
new_data = new_data.copy() 
new_data['title'] = new_data['title'].astype('string')

print(new_data['title'])


# bag of words model, count frequency of words 

from sklearn.feature_extraction.text import CountVectorizer

import numpy as np 

x = np.array(new_data.iloc[:,0].values)
y = np.array(new_data.subredditId.values)
cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(new_data.title).toarray()
print("X.shape = ",x.shape)
print("y.shape = ",y.shape)


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 40)
print(len(x_train))
print(len(x_test))


from sklearn.metrics import classification_report, accuracy_score

# rbf and linear kernels 

0        Whoever created the tradition of not seeing th...
1        We laugh at dogs getting excited when they hea...
2        When you're a kid, you don't realize you're al...
3        Dads probably bond with dogs so much because, ...
4        Being able to tolerate the sound of your own v...
                               ...                        
10934           I am 22 and will die in 2 months - 2 years
10935    I'm a dad who quit his job to make board games...
10936    IamA 102 year old man, former chicken farmer, ...
10937    I’m Brett Murphy, a ProPublica reporter who ju...
10938                     STEVE-O HERE! ASK ME ANYTHING!!!
Name: title, Length: 10889, dtype: string
X.shape =  (10889, 5000)
y.shape =  (10889,)
7622
3267


In [ ]:
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

#print("Classification Report:\n")
#print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")


svm_model = SVC(kernel='linear')
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

#print("Classification Report:\n")
#print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")

In [ ]:
x_train_rbf = x_train 
y_train_rbf = y_train 

svm_model = SVC(kernel = 'rbf', C = 0.5)
svm_model.fit(x_train_rbf, y_train_rbf)
y_pred_rbf = svm_model.predict(x_test_rbf)


In [16]:
svm_model = SVC(kernel='rbf', C=1)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")



Classification Report:

              precision    recall  f1-score   support

           0       0.53      0.85      0.65       289
           1       0.45      0.41      0.42       291
           2       0.25      0.15      0.18       296
           3       0.41      0.41      0.41       287
           4       0.27      0.29      0.28       294
           5       0.65      0.49      0.56       297
           6       0.45      0.32      0.37       293
           7       1.00      0.98      0.99       307
           8       0.33      0.63      0.43       314
           9       0.51      0.29      0.37       291
          10       0.97      0.84      0.90       308

    accuracy                           0.52      3267
   macro avg       0.53      0.51      0.51      3267
weighted avg       0.53      0.52      0.51      3267

Accuracy: 51.73%


In [17]:
svm_model = SVC(kernel='poly', C=1)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")

Classification Report:

              precision    recall  f1-score   support

           0       0.65      0.48      0.55       289
           1       0.12      0.98      0.22       291
           2       0.38      0.05      0.08       296
           3       0.26      0.07      0.11       287
           4       0.24      0.03      0.06       294
           5       0.67      0.19      0.29       297
           6       0.58      0.05      0.09       293
           7       0.99      0.88      0.93       307
           8       0.00      0.00      0.00       314
           9       0.47      0.09      0.16       291
          10       0.98      0.56      0.71       308

    accuracy                           0.31      3267
   macro avg       0.49      0.31      0.29      3267
weighted avg       0.49      0.31      0.29      3267

Accuracy: 30.82%


In [18]:
svm_model = SVC(kernel='poly', C=0.5)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")

Classification Report:

              precision    recall  f1-score   support

           0       0.65      0.44      0.53       289
           1       0.12      0.99      0.21       291
           2       0.32      0.03      0.06       296
           3       0.28      0.05      0.08       287
           4       0.07      0.01      0.02       294
           5       0.62      0.12      0.21       297
           6       0.71      0.04      0.08       293
           7       0.98      0.77      0.86       307
           8       0.00      0.00      0.00       314
           9       0.49      0.08      0.14       291
          10       0.98      0.50      0.67       308

    accuracy                           0.28      3267
   macro avg       0.47      0.28      0.26      3267
weighted avg       0.48      0.28      0.26      3267

Accuracy: 27.61%


In [7]:
svm_model = SVC(kernel='rbf', C=0.5)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")



Classification Report:

              precision    recall  f1-score   support

           0       0.46      0.87      0.60       289
           1       0.44      0.36      0.40       291
           2       0.22      0.11      0.15       296
           3       0.39      0.37      0.38       287
           4       0.29      0.30      0.30       294
           5       0.59      0.39      0.47       297
           6       0.39      0.28      0.32       293
           7       1.00      0.96      0.98       307
           8       0.31      0.66      0.43       314
           9       0.49      0.23      0.32       291
          10       1.00      0.77      0.87       308

    accuracy                           0.49      3267
   macro avg       0.51      0.48      0.47      3267
weighted avg       0.51      0.49      0.48      3267

Accuracy: 48.7%


In [14]:
svm_model = SVC(kernel='rbf', C=0.2)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")


Classification Report:

              precision    recall  f1-score   support

           0       0.38      0.87      0.53       289
           1       0.31      0.54      0.40       291
           2       0.22      0.09      0.13       296
           3       0.36      0.29      0.32       287
           4       0.26      0.24      0.25       294
           5       0.50      0.30      0.38       297
           6       0.32      0.26      0.29       293
           7       1.00      0.95      0.97       307
           8       0.34      0.53      0.42       314
           9       0.46      0.16      0.24       291
          10       0.99      0.59      0.74       308

    accuracy                           0.44      3267
   macro avg       0.47      0.44      0.42      3267
weighted avg       0.47      0.44      0.43      3267

Accuracy: 44.11%


In [15]:
svm_model = SVC(kernel='linear', C=0.5)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")

Classification Report:

              precision    recall  f1-score   support

           0       0.72      0.73      0.72       289
           1       0.42      0.48      0.44       291
           2       0.21      0.22      0.21       296
           3       0.38      0.37      0.37       287
           4       0.30      0.35      0.32       294
           5       0.61      0.56      0.58       297
           6       0.52      0.34      0.41       293
           7       1.00      0.99      1.00       307
           8       0.38      0.52      0.44       314
           9       0.50      0.38      0.43       291
          10       0.98      0.90      0.94       308

    accuracy                           0.54      3267
   macro avg       0.55      0.53      0.54      3267
weighted avg       0.55      0.54      0.54      3267

Accuracy: 53.54%


In [19]:
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")


svm_model = SVC(kernel='linear', C=0.2)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")

Classification Report:

              precision    recall  f1-score   support

           0       0.71      0.72      0.71       289
           1       0.39      0.48      0.43       291
           2       0.20      0.20      0.20       296
           3       0.34      0.36      0.35       287
           4       0.29      0.34      0.31       294
           5       0.59      0.55      0.57       297
           6       0.45      0.31      0.37       293
           7       1.00      0.99      1.00       307
           8       0.37      0.43      0.40       314
           9       0.44      0.36      0.40       291
          10       0.98      0.90      0.94       308

    accuracy                           0.52      3267
   macro avg       0.52      0.51      0.52      3267
weighted avg       0.53      0.52      0.52      3267

Accuracy: 51.61%
Classification Report:

              precision    recall  f1-score   support

           0       0.72      0.73      0.73       289
           1 

In [26]:
svm_model = SVC(kernel='poly', C=0.2)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy}%")

Classification Report:

              precision    recall  f1-score   support

           0       0.70      0.37      0.48       289
           1       0.11      1.00      0.19       291
           2       0.44      0.03      0.05       296
           3       0.31      0.04      0.07       287
           4       0.00      0.00      0.00       294
           5       0.59      0.05      0.10       297
           6       0.70      0.02      0.05       293
           7       1.00      0.51      0.68       307
           8       0.00      0.00      0.00       314
           9       0.48      0.05      0.09       291
          10       0.97      0.38      0.55       308

    accuracy                           0.22      3267
   macro avg       0.48      0.22      0.21      3267
weighted avg       0.48      0.22      0.21      3267

Accuracy: 22.31%


/Users/emilywu/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/emilywu/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/emilywu/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print(confusion_matrix(y_test, y_pred))

len(y_test)

print(confusion_matrix(y_train, y_train))

[[250   4   3   8   1   6  13   0   3   1   0]
 [  3 106  12   8  20   6   4   0 123   9   0]
 [ 18  21  34  41  36  15  15   0 100  15   1]
 [ 35  10  25 107  35  18  23   0  25   9   0]
 [ 16  17   9  21  88   7  21   0 101  14   0]
 [ 61  16  18  22  15 115  24   0  20   6   0]
 [ 63   7  13  35  29  13  81   0  39  13   0]
 [  7   0   0   1   0   1   2 296   0   0   0]
 [  5  41  20   1  31   4   1   0 208   3   0]
 [ 43  18  22  27  42  10  20   0  41  68   0]
 [ 48   1   0   6   5   1   6   0   3   0 238]]
[[674   0   0   0   0   0   0   0   0   0   0]
 [  0 696   0   0   0   0   0   0   0   0   0]
 [  0   0 692   0   0   0   0   0   0   0   0]
 [  0   0   0 709   0   0   0   0   0   0   0]
 [  0   0   0   0 702   0   0   0   0   0   0]
 [  0   0   0   0   0 699   0   0   0   0   0]
 [  0   0   0   0   0   0 705   0   0   0   0]
 [  0   0   0   0   0   0   0 687   0   0   0]
 [  0   0   0   0   0   0   0   0 667   0   0]
 [  0   0   0   0   0   0   0   0   0 702   0]
 [  0   0   

In [21]:
x_train_rbf = x_train 
y_train_rbf = y_train 

svm_model = SVC(kernel = 'rbf', C = 0.5)
svm_model.fit(x_train_rbf, y_train_rbf)
y_pred_rbf = svm_model.predict(x_test)



x_train_linear = x_train 
y_train_linear = y_train 

svm_model = SVC(kernel = 'linear', C = 0.5)
svm_model.fit(x_train_linear, y_train_linear)
y_pred_linear = svm_model.predict(x_test)


x_train_poly = x_train 
y_train_poly = y_train 

svm_model = SVC(kernel = 'poly', C = 0.5)
svm_model.fit(x_train_poly, y_train_poly)
y_pred_poly = svm_model.predict(x_test)




In [31]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("RBF confusion matrix:")
print(confusion_matrix(y_test, y_pred_rbf))

print("")

print("Linear confusion matrix:")

print("")

print(confusion_matrix(y_test, y_pred_linear))
print("")


print("")

print(confusion_matrix(y_train, y_train_rbf))



print(confusion_matrix(y_train, y_train_linear))

RBF confusion matrix:
[[250   4   3   8   1   6  13   0   3   1   0]
 [  3 106  12   8  20   6   4   0 123   9   0]
 [ 18  21  34  41  36  15  15   0 100  15   1]
 [ 35  10  25 107  35  18  23   0  25   9   0]
 [ 16  17   9  21  88   7  21   0 101  14   0]
 [ 61  16  18  22  15 115  24   0  20   6   0]
 [ 63   7  13  35  29  13  81   0  39  13   0]
 [  7   0   0   1   0   1   2 296   0   0   0]
 [  5  41  20   1  31   4   1   0 208   3   0]
 [ 43  18  22  27  42  10  20   0  41  68   0]
 [ 48   1   0   6   5   1   6   0   3   0 238]]

Linear confusion matrix:

[[210   8   6  12  11  17  11   0   8   5   1]
 [  2 139  29   6  34   3   6   0  67   5   0]
 [  8  36  64  40  43  17  13   0  54  21   0]
 [  9  17  50 105  33  19  14   0  18  20   2]
 [  9  21  28  24 104  10  14   0  55  29   0]
 [ 14  10  32  16  23 167  16   0  10   7   2]
 [ 23  12  29  36  32  16 101   0  28  16   0]
 [  0   0   0   0   0   1   0 305   1   0   0]
 [  5  60  27   9  28  10   1   0 164  10   0]
 [  6  28 